In [3]:
import tensorflow as tf
import numpy as np
import os
from utils import *
import pickle
from model import ImageCaptioningModel
from sklearn.model_selection import train_test_split
from dataset import DataGenerator
import warnings
warnings.filterwarnings("ignore")


tf.get_logger().setLevel('ERROR')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [4]:
CWD = os.getcwd()
BATCH_SIZE = 16

In [5]:
embeddings_path = os.path.join(CWD, 'glove.6B/glove.6B.100d.txt')
embeddings_index = load_embeddings(embeddings_path)
embedding_dim = 100
captions_path = os.path.join(CWD, 'data/captions.txt')
caption_dict = preprocess_captions(captions_path)
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
vocab = get_vocab(caption_dict)
tokenizer.fit_on_texts(vocab)
vocab_size = len(vocab) + 1
embedding_matrix = create_embedding_matrix(tokenizer, vocab_size, embeddings_index, embedding_dim)
max_seq_length = get_max_length(caption_dict)

In [7]:
features_path = os.path.join(CWD, 'encodings.pkl')
with open(features_path, 'rb') as f:
    features_dict = pickle.load(f)

In [8]:
embedding_matrix.shape

(4083, 100)

In [9]:
model = ImageCaptioningModel(vocab_size, max_seq_length, embedding_matrix, embedding_dim)

In [10]:
keys = list(caption_dict.keys())
train_keys, test_keys = train_test_split(keys, test_size = 0.2, random_state = 42)
train_keys, val_keys = train_test_split(train_keys, test_size = 0.25, random_state = 42)

In [11]:
train_captions = {k: caption_dict[k] for k in train_keys}
val_captions = {k: caption_dict[k] for k in val_keys}
test_captions = {k: caption_dict[k] for k in test_keys}

train_features = {k: features_dict[k] for k in train_keys}
val_features = {k: features_dict[k] for k in val_keys}
test_features = {k: features_dict[k] for k in test_keys}


In [12]:
train_data_generator = DataGenerator(train_captions, train_features, tokenizer, max_seq_length, vocab_size, batch_size=BATCH_SIZE)
val_data_generator = DataGenerator(val_captions, val_features, tokenizer, max_seq_length, vocab_size, batch_size=BATCH_SIZE)
test_data_generator = DataGenerator(test_captions, test_features, tokenizer, max_seq_length, vocab_size, batch_size=BATCH_SIZE)

In [14]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
# model = tf.keras.models.load_model(os.path.join(CWD, 'model'))

In [12]:
model_path = os.path.join(CWD, "model")
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=0)

# earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta = 0, patience = 5, verbose = 0, restore_best_weights=True)

# learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
#                                             patience=3, 
#                                             verbose=0, 
#                                             factor=0.5, 
#                                             min_lr=1e-6)

In [13]:
NUM_EPOCHS = 50
with tf.device('/cpu:0'):
   history = model.fit(train_data_generator,
                     epochs = NUM_EPOCHS,
                  validation_data = val_data_generator,
                  callbacks = [checkpoint])

Epoch 1/50
304/304 [==============================] - 150s 489ms/step - loss: 5.0047 - accuracy: 0.1772 - val_loss: 4.2682 - val_accuracy: 0.2400
Epoch 2/50
304/304 [==============================] - 166s 547ms/step - loss: 4.1026 - accuracy: 0.2413 - val_loss: 3.9076 - val_accuracy: 0.2664
Epoch 3/50
304/304 [==============================] - 171s 563ms/step - loss: 3.7833 - accuracy: 0.2637 - val_loss: 3.7358 - val_accuracy: 0.2835
Epoch 4/50
304/304 [==============================] - 175s 575ms/step - loss: 3.5747 - accuracy: 0.2799 - val_loss: 3.6406 - val_accuracy: 0.2972
Epoch 5/50
304/304 [==============================] - 171s 563ms/step - loss: 3.4164 - accuracy: 0.2920 - val_loss: 3.5808 - val_accuracy: 0.3041
Epoch 6/50
304/304 [==============================] - 177s 582ms/step - loss: 3.2951 - accuracy: 0.3007 - val_loss: 3.5752 - val_accuracy: 0.3064
Epoch 7/50
304/304 [==============================] - 191s 629ms/step - loss: 3.1890 - accuracy: 0.3096 - val_loss: 3.5530 -